In [ ]:
#!flask/bin/python

import csv
import json

import sqlfile
import data_serv
import predict_serv

from flask import Flask
from flask import url_for
from flask import jsonify
from flask import abort
from flask import render_template
from flask import make_response


app = Flask(__name__)

# набор возможных действий
help = [
    {
        'title': 'info_avg_default()',
        'description': 'stub http-request to provide data for a specific session_id in absolute values. you need to '
                       'replace the default with the required session_id',
        'uri': 'http://localhost:5000/info/avg/default'
    },
    {
        'title': 'info_percent_default()',
        'description': 'stub http-request to provide data for a specific session_id in percent. you need to '
                       'replace the default with the required session_id',
        'uri': 'http://localhost:5000/info/percent/default'
    },
    {
        'title': 'get_info_with_avg(session_id)',
        'description': 'returns the number of views in each category within the session_id in ABSOLUTE VALUES',
        'uri': 'http://localhost:5000/info/avg/<string:session_id>'
    },
    {
        'title': 'get_info_with_percent(session_id)',
        'description': 'returns the number of views in each category within the session_id in PERCENT',
        'uri': 'http://localhost:5000/info/percent/<string:session_id>'
    },
    {
        'title': 'predict_gender(session_id)',
        'description': 'Predicting gender for a user by session_id',
        'uri': 'http://localhost:5000/predict/<string:session_id>'
    },
    {
        'title': 'predict_gender_with_avg(session_id):',
        'description': 'Predicting gender for a user by session_id with information about the sites he visited in '
                        'absolute values',
        'uri': 'http://localhost:5000/predict/avg/<string:session_id>'
    },
    {
        'title': 'predict_gender_with_percent(session_id):',
        'description': 'Predicting gender for a user by session_id with information about the sites he visited in '
                        'percentage',
        'uri': 'http://localhost:5000/predict/percent/<string:session_id>'
    }
]


# проверка корректности ввода session_id
def check_input(ses_id):
    if ses_id[0] != 'u':
        print('Ошибка!!! Некорректный session_id! Попробуйте ещё раз.')
        return True
    elif ses_id[1:] > '40000':
        print('Ошибка!!! Слишком большой session_id, такой не существует! Попробуйте ещё раз.')
        return True
    elif ses_id[1:] < '10001':
        print('Ошибка!!! Слишком маленький session_id, такой не существует! Попробуйте ещё раз.')
        return True

    return False


# для дальнейшего пополнения списка с функционалом
def make_public_task(help):
    new_task = {}
    for field in help:
        if field == 'id':
            new_task['uri'] = url_for('get_task', task_id=help['id'], _external=True)
        else:
            new_task[field] = help[field]
    return new_task


@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': '404! Not found'}), 404)


# ГЛАВНАЯ СТРАНИЦА
@app.route('/')
def index():
    return render_template('main_page.html')


# вывод имеющегося функционала
@app.route('/help', methods=['GET'])
def get_tasks():
    return jsonify({'help': help})


# ====================================================================================== #
#                                  Сервис доступа к данным                               #
# ====================================================================================== #
@app.route('/default/info/avg', methods=['GET'])
def info_avg_default():
    return 'Замените /default на нужный Вам /"session_id".' \
           ' (!Напоминание: корректный вид session_id: с u10001 до u40000!)'


@app.route('/default/info/percent', methods=['GET'])
def info_percent_default():
    return 'Замените /default на нужный Вам /"session_id". ' \
           '(!Напоминание: корректный вид session_id: с u10001 до u40000!)'


@app.route('/<string:session_id>/info/percent', methods=['GET'])
def get_info_with_percent(session_id):
    if check_input(session_id):
        abort(404)
    sqlfile.get_info_about_one_session(session_id)
    data = data_serv.get_data(session_id, version='percent')
    return str(data)


@app.route('/<string:session_id>/info/avg', methods=['GET'])
def get_info_with_avg(session_id):
    if check_input(session_id):
        abort(404)
    sqlfile.get_info_about_one_session(session_id)
    data = data_serv.get_data(session_id, version='avg')
    return str(data)


# ====================================================================================== #
#                               Работа с предиктивным сервисом                           #
# ====================================================================================== #
@app.route('/<string:session_id>/predict', methods=['GET'])
def predict_gender(session_id):
    if check_input(session_id):
        abort(404)
    return predict_serv.input_session_id_and_predict_gender(session_id)


@app.route('/<string:session_id>/predict/avg', methods=['GET'])
def predict_gender_with_avg(session_id):
    if check_input(session_id):
        abort(404)
    return predict_serv.input_session_id_and_predict_gender(session_id, version='avg')


@app.route('/<string:session_id>/predict/percent', methods=['GET'])
def predict_gender_with_percent(session_id):
    if check_input(session_id):
        abort(404)
    return predict_serv.input_session_id_and_predict_gender(session_id, version='percent')


if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
